In [17]:
# Normal stack of pandas, numpy, matplotlib and seaborn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns

import os
import json

# Factual API
from factual import Factual # https://github.com/Factual/factual-python-driver
from factual.utils import circle

%matplotlib inline

Factual API TODOs in order to make it work on your localhost:
 1. run in terminal "pip install factual-api" without ""
 2. sign up for a Factual API key here: https://www.factual.com/contact/new#free_api_access
 3. Set up your key and secret as environment variables - FACTUAL_API_KEY and FACTUAL_API_SECRET respectively
    
    3.1. On mac you just open .bash_profile and add two lines: 
     
     export FACTUAL_API_KEY=your_key
     
     export FACTUAL_API_SECRET=your_secret
     
    3.2. Restart shell and run jupyter notebook in this newly restarted shell (terminal)
     
 4. ...profit ;)

In [5]:
# You can verify that your env variables are ok with:
# print(os.environ['FACTUAL_API_KEY'])
# print(os.environ['FACTUAL_API_SECRET'])

In [6]:
factual = Factual(os.environ['FACTUAL_API_KEY'], os.environ['FACTUAL_API_SECRET'])

In [9]:
places = factual.table('places-ch')
places.schema()

{'description': 'Businesses and Places of Interest',
 'fields': [{'datatype': 'String',
   'description': 'Business/POI name',
   'faceted': False,
   'label': 'Name',
   'multivalued': False,
   'name': 'name',
   'q_searchable': True,
   'searchable': True,
   'sortable': True,
   'writable': True},
  {'datatype': 'String',
   'description': 'Address number and street name',
   'faceted': False,
   'label': 'Address',
   'multivalued': False,
   'name': 'address',
   'q_searchable': True,
   'searchable': True,
   'sortable': True,
   'writable': True},
  {'datatype': 'String',
   'description': 'Additional address, incl. suite numbers',
   'faceted': False,
   'label': 'Address Extended',
   'multivalued': False,
   'name': 'address_extended',
   'q_searchable': False,
   'searchable': True,
   'sortable': False,
   'writable': True},
  {'datatype': 'String',
   'description': 'PO Box. As they do not represent the physical location of a brick-and-mortar store, PO Boxes are often exc

In [10]:
places.data()

[{'address': 'Binningerstrasse 40',
  'category_ids': [371],
  'category_labels': [['Social', 'Zoos, Aquariums and Wildlife Sanctuaries']],
  'country': 'ch',
  'factual_id': '6aa834ab-8ed1-459a-93ea-4ce2223c26ae',
  'fax': '061 281 00 05',
  'hours': {'friday': [['8:00', '17:00']],
   'monday': [['5:00', '7:00'], ['10:00', '15:00']],
   'saturday': [['9:00', '16:00']],
   'sunday': [['8:00', '17:00']],
   'thursday': [['10:00', '15:00']],
   'tuesday': [['6:00', '7:00'], ['10:00', '16:00']],
   'wednesday': [['11:00', '16:00']]},
  'hours_display': 'Mon 5:00-7:00, 10:00-15:00; Tue 6:00-7:00, 10:00-16:00; Wed 11:00-16:00; Thu 10:00-15:00; Fri 8:00-17:00; Sat 9:00-16:00; Sun 8:00-17:00',
  'latitude': 47.547416,
  'locality': 'Basel',
  'longitude': 7.578764,
  'name': 'Zoo Basel',
  'postcode': '4054',
  'region': 'Basel-Stadt',
  'tel': '061 295 35 35',
  'website': 'http://www.zoobasel.ch/'},
 {'address': 'Grossmünsterplatz',
  'category_ids': [55],
  'category_labels': [['Community 

In [18]:
#Coffe near EPFL
epfl_coffe = places.search('coffee').geo(circle(46.5198, 6.5657, 1000)).data()
print(json.dumps(epfl_coffe, indent=2))

[
  {
    "website": "http://lesplanade-ecublens-ch.placestars.com",
    "tel": "021 693 32 58",
    "$distance": 47.60073,
    "name": "L'esplanade",
    "country": "ch",
    "factual_id": "c9b2e7b5-fc3c-491c-aded-03d35ad4f761",
    "longitude": 6.565408,
    "region": "Vaud",
    "hours": {
      "thursday": [
        [
          "7:00",
          "20:00"
        ]
      ],
      "wednesday": [
        [
          "7:00",
          "20:00"
        ]
      ],
      "tuesday": [
        [
          "7:00",
          "20:00"
        ]
      ],
      "monday": [
        [
          "7:00",
          "20:00"
        ]
      ],
      "saturday": [
        [
          "8:00",
          "15:00"
        ]
      ],
      "friday": [
        [
          "7:00",
          "20:00"
        ]
      ]
    },
    "latitude": 46.520178,
    "category_ids": [
      342,
      347
    ],
    "locality": "Lausanne",
    "hours_display": "Mon-Fri 7:00-20:00; Sat 8:00-15:00",
    "category_labels": [
     

TODOs:
 - We gotta figure out how we would like to divide Switzerland into small pieces (squares or circles) such that we will be able to extract all spots provided by factual without actually hitting its limit ("3 Page limit is the maximum number of rows that can be returned in a single request using the limit parameter. Row limit is the maximum depth a request can page into using (e.g., offset + limit)."